In [1]:
# from time import sleep
# from random import randint
from bs4 import BeautifulSoup
# import datetime
import requests
import pandas as pd
import re

In [2]:
def get_page_source(url):
    agent = {"User-Agent":"Mozilla/5.0"} # User Agent defined to avoid robot detection
    source=requests.get(url, headers=agent)
    return source.text, source.status_code

def get_soup(content):
    soup=BeautifulSoup(content, "lxml")
    return soup

In [3]:
def get_links(url = "https://www.tradinghours.com/exchanges", 
              base_url = "https://www.tradinghours.com"):
    text, status = get_page_source(url)
    soup = get_soup(text)

    # First scrape all links
    table = soup.find('table', attrs = {'id' : 'exchangetable',
                                        'class' : 'table table-striped table-bordered table-hover'})

    links = table.findAll('a')
    links_all = []
    
    for link in links:
        link = link.get('href')
        if  'market-holidays' not in link and 'trading-hours' not in link:
            links_all.append(base_url+ "/" + link)
    
    return links_all

In [4]:
def get_details_exchange(link):
    text, status = get_page_source(link)
    soup = get_soup(text)
    # Name of Stock Exchange
    title = soup.find('h1', attrs = {'class':'heading mt-0'}).text.strip()

    #Address
    spans = soup.find('div', attrs = {'class' : 'card-body'}).find(
        'div', attrs = {'class' : 'row text-center'}).findAll('span')
    address = spans[1].text.strip()

    #Contact
    contact_text = soup.find('div', class_ = 'col-md-8').findAll('li')
    contact_number = contact_text[0].text.strip()
    contact_number = contact_number[contact_number.find('+'):]
    
    # Full Adress : 
    link = link+'/contact' # Add /contact for full address
    text, status = get_page_source(link)
    soup = get_soup(text)
    para = soup.find('address')
    full_address = []
    contents = para.contents
    for content in contents:
        text = str(content.string)
        if 'None' not in text:
            full_address.append(text.strip())
            
            
    return {'Name' : title, 
       'Address' : address,
       'Contact' : contact_number,
       'Full Address' : full_address}    

In [5]:
def final_data():
    result = list()
    links = get_links()
    for link in links:
        result.append(get_details_exchange(link))
    return pd.DataFrame(result)

In [6]:
df = final_data()
df

,Name,Address,Contact,Full Address
0,New York Stock Exchange,"New York, United States",+1 212-896-2830,"[11 Wall Street, New York, NY 10005]"
1,NASDAQ Stock Exchange,"New York, United States",+1 212-401-8700,"[One Liberty Plaza, 165 Broadway, New York, NY..."
2,London Stock Exchange,"London, United Kingdom",+44 20 7797 4400,"[10 Paternoster Square, London, EC4M 7LS]"
3,Tokyo Stock Exchange,"Tokyo, Japan",+81 3-3666-1361,"[2-1 Nihombashi Kabutocho, Chuo-ku, Tokyo 103-..."
4,Shanghai Stock Exchange,"Shanghai, China",+86 21 6880 8888,"[China, Shanghai Shi, Pudong Xinqu, Lu Jia Zui..."
...,...,...,...,...
139,Sydney Stock Exchange,"Sydney, Australia",+61 2 9217 2730,"[Level 41, 259 George Street, Sydney NSW 2000]"
140,Belarusian Currency and Stock Exchange,"Minsk, Belarus",+375 17 309-33-22,"[48a,Surganova str., 220013 Minsk, Belarus]"
141,Korea Exchange (KOSDAQ),"Seoul, South Korea",+82 1577-0088,"[76, Yeouinaru-ro, Yeongdeungpo-gu, Seoul]"
142,Regional Securities Exchange SA,"Abidjan, Ivory Coast",+225 20 32 66 84,"[BRVM - Regional Stock Exchange, 18, Rue Josep..."


In [57]:
# Getting details from Address
# Assumption : PIN COde is atleast 3 digits
def get_pincode(text):
    a = re.findall("[1-9][0-9][0-9][0-9]+", text[-1])
    if(len(a)): return a[0]
    
df['State'] = df['Address'].apply(lambda x : re.split(',', x)[0])
df['Country'] = df['Address'].apply(lambda x : re.split(',', x)[-1])
df['Pincode'] = df['Full Address'].apply(lambda x : get_pincode(x))

df.to_csv("trading_hours_exchanges.csv")
df

,Name,Address,Contact,Full Address,State,Country,Pincode
0,New York Stock Exchange,"New York, United States",+1 212-896-2830,"[11 Wall Street, New York, NY 10005]",New York,United States,10005
1,NASDAQ Stock Exchange,"New York, United States",+1 212-401-8700,"[One Liberty Plaza, 165 Broadway, New York, NY...",New York,United States,10006
2,London Stock Exchange,"London, United Kingdom",+44 20 7797 4400,"[10 Paternoster Square, London, EC4M 7LS]",London,United Kingdom,None
3,Tokyo Stock Exchange,"Tokyo, Japan",+81 3-3666-1361,"[2-1 Nihombashi Kabutocho, Chuo-ku, Tokyo 103-...",Tokyo,Japan,8224
4,Shanghai Stock Exchange,"Shanghai, China",+86 21 6880 8888,"[China, Shanghai Shi, Pudong Xinqu, Lu Jia Zui...",Shanghai,China,None
...,...,...,...,...,...,...,...
139,Sydney Stock Exchange,"Sydney, Australia",+61 2 9217 2730,"[Level 41, 259 George Street, Sydney NSW 2000]",Sydney,Australia,2000
140,Belarusian Currency and Stock Exchange,"Minsk, Belarus",+375 17 309-33-22,"[48a,Surganova str., 220013 Minsk, Belarus]",Minsk,Belarus,220013
141,Korea Exchange (KOSDAQ),"Seoul, South Korea",+82 1577-0088,"[76, Yeouinaru-ro, Yeongdeungpo-gu, Seoul]",Seoul,South Korea,None
142,Regional Securities Exchange SA,"Abidjan, Ivory Coast",+225 20 32 66 84,"[BRVM - Regional Stock Exchange, 18, Rue Josep...",Abidjan,Ivory Coast,3802
